<img src="../demos/FIUM.png" width="350px" class="pull-right" style="display: inline-block">

# Visión Artificial

### 4º de Grado en Ingeniería Informática

Curso 2020-2021<br>
Convocatoria de Julio<br>
Marisol Isabel Zucca

In [25]:
import cv2 as cv
import numpy as np
import math
import matplotlib.path as mpltPath

# Ejercicios

## Ejercicio 1 - CALIBRACIÓN

#### a) Realiza una calibración precisa de tu cámara mediante múltiples imágenes de un chessboard.

Para poder realizar una calibración precisa de mi cámara (en mi caso, la del móvil), vamos a apoyarnos en el siguiente chessboard y la aplicación <i>calibrate.py</i> proporcionada por el profesorado.

<img src="../images/calibracion/pattern.png" width="400px"><br>

Será necesario tomar varias fotos del marcador con distintas perspectivas, y a continuación ejecutar <i>calibrate.py</i> para que analice las fotos y obtenga la siguiente matriz:<br>

<p><i>
RMS: 3.217471748824684<br>
camera matrix:<br>
 [[3.33350026e+03 0.00000000e+00 1.37895391e+03]<br>
 [0.00000000e+00 3.30693258e+03 2.09307524e+03]<br>
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]<br>
distortion coefficients:  [ 0.21264965 -1.51913124 -0.0067358  -0.02069358  2.10661863]<br>
</i></p>

Observando el valor en la primera fila y columna, obtenemos la distancia focal precisa:

In [5]:
f = 3333.50026
f

3333.50026

#### b) Haz una calibración aproximada con un objeto de tamaño conocido y compara con el resultado anterior.

Para realizar la calibración aproximada, tomaremos una foto de un objeto de tamaño conocido a una distancia conocida. A continuación mediremos el tamaño del objeto en píxeles y aplicaremos la siguiente fórmula para obtener la distancia focal aproximada:

$$u = f \frac{X}{Z}$$

Siendo $u$ y $X$ la altura en píxeles y cm respectivamente del objeto observado, y $Z$ la distancia de la cámara, también en cm.<br>
La siguiente foto ha sido tomada a 60cm de distancia y la altura del libro es de 18cm.

<img src="../images/calibracion/b/libro.jpg" width="350px"><br>

Si recortamos la foto al filo, obtenemos un tamaño de 570x924, por lo que la altura de 18cm equivale a 924px.

<img src="../images/calibracion/b/libro18cm.jpg" width="350px"><br>

Aplicamos entonces la fórmula para obtener la distancia focal aproximada:

$$f = u \frac{Z}{X}$$

In [6]:
Z = 60 # distancia cámara en cm
X_book = 18 # altura en cm
u_book = 924 # altura en px

f_book = u_book * Z / X_book
f_book

3080.0

Vamos a realizar otra prueba con un segundo objeto: la distancia será la misma, pero la Nintendo Switch Lite mide 21cm en altura.

<img src="../images/calibracion/b/switch.jpg" width="350px"><br>

En este caso el valor de 21cm equivale a 1031px.

<img src="../images/calibracion/b/switch21cm.jpg" width="350px">

In [8]:
Z = 60 # distancia cámara en cm
X_switch = 21 # altura en cm
u_switch = 1031 # altura en px

f_switch = u_switch * Z / X_switch
f_switch

2945.714285714286

Hemos obtenidos unos valores de distancia focal de 3080 y 2945.7 respectivamente. El valor preciso es 3333.5, que es un valor parecido ($\pm 30\%$) a los obtenidos de forma aproximada. Los resultados son, por tanto, correctos.

#### c) Determina a qué altura hay que poner la cámara para obtener una vista cenital completa de un campo de baloncesto.

Una vez más, podemos aplicar la fórmula anteriormente usada, pero teniendo la Z (la distancia de la cámara) como incógnita:

$$Z = f \frac{X}{u}$$

Las medidas oficiales de un campo de basket son 28 metros de largo y 15 metros de ancho.

<img src="../images/basket.gif"><br>

Tomaremos la medida más grande como nuestra $X$. La resolución de la cámara que se está usando es de 3000 x 4000, por lo que tomaremos $u = 4000$. Finalmente, usaremos como f el valor de la calibración precisa.

In [11]:
width = 3000
height = 4000

X_basket = 28 * 100 # altura en cm
u_basket = height # altura en px

Z_basket = f * X_basket / u_basket
Z_basket

2333.450182

Hemos obtenido un valor de 2333.5cm, es decir, para obtener una vista cenital completa de un campo de baloncesto de 28m de largo, será necesario posicionar nuestra cámara a 23,34m de altura.

#### d) Haz una aplicación para medir el ángulo que definen dos puntos marcados con el ratón en el imagen.

La aplicación implementada está basada en parte en el código de <i>medidor.py</i>, del cual se ha obtenido la forma de marcar puntos con el ratón en la imagen.

Tras marcar dos puntos en la imagen, obtendremos sus coordenadas y llamaremos a una función para aplicar la siguiente fórmula sobre los vectores obtenidos a partir de los dos puntos marcados y obtener así el ángulo que nos interesa:

$$u \times v = |u||v|cos(\alpha)$$

La función implementada es la siguiente:

In [12]:
def angle(p1, p2):
    u = [ p1[0]-WIDTH/2, p1[1]-HEIGHT/2, f]
    v = [ p2[0]-WIDTH/2, p2[1]-HEIGHT/2, f]

    # cos(alpha) = u . v / (|u| * |v|) 
    cosAlpha = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

    return math.degrees( math.acos(cosAlpha) )

Los puntos $p1$ y $p2$ se le pasan como tuplas (x,y). A partir de estos dos puntos se obtendrán los vectores $u$ y $v$ (basándonos en el centro del imagen de ancho $WIDTH$ y alto $HEIGHT$), que son los que se usarán para, mediante el uso de funciones de las librerias numpy y math, obtener el ángulo buscado.

A continuación vemos una demostración de funcionamiento.

<img src="../demos/images/calibracion.bmp">

#### e) Opcional: determina la posición aproximada desde la que se ha tomado una foto a partir ángulos observados respecto a puntos de referencia conocidos.

## Ejercicio 2 - ACTIVIDAD

#### Construye un detector de movimiento en una región de interés de la imagen marcada manualmente. Guarda 2 ó 3 segundos de la secuencia detectada en un archivo de vídeo. Opcional: muestra el objeto seleccionado anulando el fondo.

<img src="../demos/images/actividad.bmp">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/actividad.mkv"></video>

## Ejercicio 3 - COLOR

#### Construye un clasificador de objetos en base a la similitud de los histogramas de color del ROI (de los 3 canales por separado). Más información. Opcional: Segmentación densa por reproyección de histograma.

<img src="../demos/images/color.bmp">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/color.mkv"></video>

## Ejercicio 4 - FILTROS

#### Muestra el efecto de diferentes filtros sobre la imagen en vivo de la webcam. Selecciona con el teclado el filtro deseado y modifica sus posibles parámetros (p.ej. el nivel de suavizado) con las teclas o con trackbars. Aplica el filtro en un ROI para comparar el resultado con el resto de la imagen. Opcional: implementa en Python o C "desde cero" algún filtro y compara la eficiencia con OpenCV.

<img src="../demos/images/filtros.bmp">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/filtros.mkv"></video>

## Ejercicio 5 - SIFT

#### Escribe una aplicación de reconocimiento de objetos (p. ej. carátulas de CD, portadas de libros, cuadros de pintores, etc.) con la webcam basada en el número de coincidencias de keypoints.

<img src="../demos/images/sift.bmp">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/sift.mkv"></video>

## Ejercicio 6 - RECTIF

#### Rectifica la imagen de un plano para medir distancias (tomando manualmente referencias conocidas). Por ejemplo, mide la distancia entre las monedas en coins.png o la distancia a la que se realiza el disparo en gol-eder.png. Verifica los resultados con imágenes originales tomadas por ti.

<img src="../demos/images/rectif.bmp">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/rectif.mkv"></video>

## Ejercicio 7 - PANO

#### Crea automáticamente un mosaico a partir de las imágenes en una carpeta. Las imágenes no tienen por qué estar ordenadas ni formar una cadena lineal y no sabemos el espacio que ocupa el resultado. El usuario debe intervenir lo menos posible. Recuerda que debe tratarse de una escena plana o de una escena cualquiera vista desde el mismo centro de proyección. Debes usar homografías. Compara el resultado con el que obtiene la utilidad de stitching de OpenCV.

<img src="../demos/images/pano_my.jpg">
<img src="../demos/images/pano_fium.jpg">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/pano.mkv"></video>

## Ejercicio 8 - RA

#### Crea un efecto de realidad aumentada interactivo.

El código de este ejercicio está basado en la aplicación <i>pose2.py</i>

#### a) Los objetos virtuales deben cambiar de forma, posición o tamaño siguiendo alguna lógica
Cuando se encuentre un marcador, se dibujará un cubo transparente sobre el mismo, que se irá moviendo recorriendo los lados del cuadrado definido por las equinas más largas del marcador. Pulsando las tecla + y - podremos cambiar las dimensiones del cubo.

#### b) El usuario puede observar la escena cambiante desde cualquier punto de vista moviendo la cámara alrededor del marcador
Esta parte del ejercicio viene implementada por el código que ya se encuentra en <i>pose2.py</i>, por lo que no ha sido necesario implementarlo individualmente.

#### c) El usuario puede marcar con el ratón en la imagen puntos del plano de la escena para interactuar con los objetos virtuales.
Haciendo click sobre el cubo en movimiento, se podrá activar o desactivar el texturizado del mismo.

Las funciones de interés que permiten el funcionamiento indicado se detallan a continuación.

### Implementación

El movimiento del cubo se hace a través del uso de las funciones seno y coseno, que permiten obtener valores entre -1 y 1 a lo largo del tiempo (dado en número de frames).

Usaremos los valores del seno para determinar la dirección en la que tenga que moverse el cubo sobre el marcador.
* Cuando el valor del seno esté creciendo del 0 al 1, el cubo tendrá que moverse hacia arriba.
* Cuando el valor del seno esté decreciendo del 1 al 0, el cubo tendrá que moverse hacia la derecha.
* Cuando el valor del seno esté decreciendo del 0 al -1, el cubo tendrá que moverse hacia abajo.
* Cuando el valor del seno esté creciendo del -1 al 0, el cubo tendrá que moverse hacia la izquierda.

In [ ]:
UD = 0 # up/down movement
LR = 0 # left/right movement
sin = 0
cos = 1
minSize = 0.25 # minimum size of the cube
maxSize = 1    # maximum size of the cube
size = minSize # actual size of the cube

pikapika = False # if True, draw Pikachu

#########################################
# Lo que sigue va en el bucle del video #
#########################################

# Move the cube along the marker
oldsin = sin
sin = np.sin(n/50)

if oldsin <= sin and sin >= 0 and sin <= 1: # up
    UD = abs(sin)
    LR = 0
elif oldsin > sin and sin >= 0 and sin <= 1: # right
    UD = 1
    LR = abs(np.cos(n/50))
elif oldsin >= sin and sin <= 0 and sin >= -1: # down
    UD = abs(np.cos(n/50))
    LR = 1
elif oldsin < sin and sin <= 0 and sin >= -1: # left
    UD = 0
    LR = abs(sin)

El control sobre si se ha hecho click sobre el cubo se hará transformando los puntos 3D del cubo en puntos 2D de la imagen y comprobando si las coordenadas (x,y) del punto marcado se encuentran dentro de las coordenadas del polígono 2D que se dibuja. Esta comprobación se hace con la función <b>contains_points()</b> de la librería <b>matplotlib.path</b>.

Cuando se hace click sobre el cubo, se pasa de un cubo transparente a un cubo texturizado y de uno texturizado a uno transparente.

In [ ]:
fig3D = cube*size + ((1-size)*LR, (1-size)*UD, 0)
fig2D = htrans(M, fig3D).astype(int)

# Check if the clicked point is inside the figure
if points[0] != (0,0):
    if mpltPath.Path(fig2D).contains_points(points):
        pikapika = not pikapika
    points[0] = (0,0)

if pikapika:
    # If the cube was clicked, draw surprised Pikachu
    drawPikachu(fig2D, frame)
else:
    # Otherwise draw a wire cube
    cv.drawContours(frame, [fig2D], -1, (0,128,0), 3, cv.LINE_AA)

La función <b>drawPikachu()</b> permite darle textura al cubo cuando se haga click sobre el mismo.
Según la perspectiva con la que se observe, de un cubo sólido será posible ver 1, 2 o como mucho 3 caras a la vez. Para que el texturizado del cubo salga correctamente, nos interesa que las caras visibles se dibujen por últimas.

Por la forma en la que funciona la detección del marcador, podemos asegurar que el cubo nunca podrá verse desde una perspectiva inferior, por lo que podemos basarnos en el valor de la $x$ de la esquina inferior izquierda de cada lado para determinar si se encuentra delante o detrás de otro lado. Un valor más alto de $x$ indica que la cara del cubo se encuentra delante de otra con un valor de $x$ menor.

Tras determinar las caras del cubo, lo que haremos será ordenarlas según el valor de $x$ de la esquina indicada anteriormente, lo que nos permitirá dibujar primero las que se encuentran detrás.

Aplicaremos <b>warpPerspective()</b> con la homografía definida por los puntos de la cara que se quiera dibujar para transformar la perspectiva de la imagen de Pikachu.

In [21]:
pikachu = cv.imread('../images/ra/pikachu.png')
h,w = pikachu.shape[:2]
src = np.array([[0,0],[0,h],[w,h],[w,0]])

def drawPikachu(cube, frame):
    sides = [
        ( cube[5][0], np.array([cube[5], cube[0], cube[1], cube[6]]) ),
        ( cube[6][0], np.array([cube[6], cube[1], cube[2], cube[7]]) ),
        ( cube[7][0], np.array([cube[7], cube[2], cube[3], cube[8]]) ),
        ( cube[8][0], np.array([cube[8], cube[3], cube[0], cube[5]]) )
    ]

    # A max of 3 sides can be seen at any given time and the cube will
    # never appear from below. The 2 sides with the highest value of x,
    # will be the ones that are actually seen. Draw them last.
    sides.sort(key=lambda s : s[0], reverse=True)

    # Top side must be the last to be drawn
    sides.append( (cube[5][0], cube[5:9]) )

    for _, dst in sides:
        H = cv.getPerspectiveTransform(src.astype(np.float32), dst.astype(np.float32))
        cv.warpPerspective(pikachu,H,(WIDTH,HEIGHT),frame,0,cv.BORDER_TRANSPARENT)

### Demostración visual

<img src="../demos/images/ra01.bmp">
<img src="../demos/images/ra02.bmp">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/ra.mkv"></video>

## Ejercicio 9 - SWAP

#### Intercambia dos cuadriláteros en una escena marcando manualmente los puntos de referencia.

Para la implementación de esta aplicación se ha vuelto a hacer uso del código <i>medidor.py</i> para la selección de puntos con el ratón.

Las funciones que permiten el funcionamiento de la aplicación son <b>mask_zone()</b> y <b>copy_quad()</b>.

Para la implementación de <b>mask_zone()</b> se ha consultado la siguiente entrada en Stack Overflow: [*Extracting polygon given coordinates from an image using OpenCV*](https://stackoverflow.com/questions/30901019/extracting-polygon-given-coordinates-from-an-image-using-opencv). Esta función permite generar una máscara poligonal a partir de ciertos puntos dados. Esa máscara se aplicará sobre la imagen original para obtener así una imagen en la que solo aparece la parte contenida en el polígono dado.

In [ ]:
def mask_zone(points):
    mask = np.zeros((frame.shape[0], frame.shape[1]))
    cv.fillConvexPoly(mask, points, 1)
    mask = mask.astype(np.bool)
    out = np.zeros_like(frame)
    out[mask] = frame[mask]
    return out, mask

La función <b>copy_quad()</b> recibirá dos cuadriláteros $p$ y $q$ definidos como 4 puntos cada uno. Generará una mascara para cada uno de ellos y la imagen obtenida tras aplicar la respectiva máscara sobre la imágen original.

A continuación hará una transformación de perspectiva de los puntos de $p$ a los de $q$ y viceversa, obteniendo las dos homografías. Con las homografías obtenidas podremos transformar la imagen contenida en $p$ para que se adapte al espacio definido por $q$ y la imagen contenida en $q$ para que se adapte al espacio definido por $p$.

Finalmente aplicamos las dos máscaras sobre la imagen original para intercambiar de sitio las dos imágenes transformadas.

In [17]:
def copy_quad(p, q, frame):

    src, mask1 = mask_zone(p)
    dst, mask2 = mask_zone(q)

    Hpq = cv.getPerspectiveTransform(p.astype(np.float32), q.astype(np.float32))
    Hqp = cv.getPerspectiveTransform(q.astype(np.float32), p.astype(np.float32))

    out1 = frame.copy()
    out2 = frame.copy()

    h,w,_ = frame.shape

    cv.warpPerspective(src,Hpq,(w,h),out1,0,cv.BORDER_TRANSPARENT)
    cv.warpPerspective(dst,Hqp,(w,h),out2,0,cv.BORDER_TRANSPARENT)

    frame[mask2] = out1[mask2]
    frame[mask1] = out2[mask1]

Para el uso correcto de la aplicación, se deberán seleccionar un total de 8 puntos, 4 por cada cuadrilatero. El primer punto por el que se empiece no es relevante, pero el orden en el que se marquen los siguientes puntos de cada cuadrilátero deberá ser coherente para ambos. Si se empieza por la esquina superior izquierda, habrá que seguir con la otra esquina superior o la inferior izquierda, y seguir seleccionando los puntos que permitan dibujar un cuadrilátero. Los puntos del segundo cuadrilátero deberán ser seleccionados en el mismo orden que los del primero.

<img src="../demos/images/swap01.bmp"><br>

Cuando se hayan marcado los 8 puntos, se intercambiarán las imágenes contenidas en ellos, aplicándole transformaciones para que se adapten al nuevo espacio. La calidad del resultado de los bordes depende del cuidado con el que se seleccionen los 8 puntos.

<img src="../demos/images/swap02.bmp">

### Video demostración

<video width="600" height="450" controls src="../demos/videos/swap.mkv"></video>

# Bibliografía

[*Extracting polygon given coordinates from an image using OpenCV*](https://stackoverflow.com/questions/30901019/extracting-polygon-given-coordinates-from-an-image-using-opencv)